In [120]:

import re
import string

#fname = 'group1_00000005-g.txt'
fname = 'C:/Users/Alienware/Desktop/ads project1/Spring2019-Proj4-grp6/data/ground_truth/group1_00000005.txt'
with open(fname) as file:
    truth = file.readlines()
truth = [x.strip() for x in truth] 
truth = [x.lower() for x in truth]
truth_str = ' '.join(truth)
for i in range(10):
    truth_str=truth_str.replace(str(i),'')
truth_clean = re.sub(r'[{}]'.format(string.punctuation), '', truth_str).split()
#truth_clean[1744:1749] = ['a','b','c','d','e']

truth_clean=truth_clean[0:1672]
truth_clean

['exhibit',
 'd',
 'staff',
 'report',
 'ma',
 'rch',
 'by',
 'william',
 'j',
 'driver',
 'congressional',
 'action',
 'involving',
 'portions',
 'of',
 'the',
 'foreign',
 'trade',
 'and',
 'investment',
 'act',
 'of',
 'suddenly',
 'loomed',
 'as',
 'a',
 'possibility',
 'last',
 'week',
 'when',
 'it',
 'was',
 'learned',
 'that',
 'the',
 'aflcio',
 'planned',
 'to',
 'attempt',
 'attaching',
 'titles',
 'vi',
 'and',
 'vii',
 'of',
 'that',
 'proposed',
 'act',
 'to',
 'the',
 'administrations',
 'gold',
 'legislation',
 'through',
 'agreement',
 'with',
 'house',
 'ways',
 'and',
 'means',
 'committee',
 'chairman',
 'wilbur',
 'd',
 'mills',
 'dark',
 'the',
 'house',
 'banking',
 'and',
 'currency',
 'committee',
 'has',
 'been',
 'given',
 'jurisdiction',
 'over',
 'titles',
 'vi',
 'and',
 'vii',
 'of',
 'the',
 'socalled',
 'the',
 'latter',
 'committee',
 'scheduled',
 'gold',
 'bill',
 'hearings',
 'march',
 'and',
 'immediately',
 'afterward',
 'is',
 'expected',
 'to',


In [112]:
#fname = 'group1_00000005-t.txt'
fname = 'C:/Users/Alienware/Desktop/ads project1/Spring2019-Proj4-grp6/data/tesseract/group1_00000005.txt'
with open(fname,encoding='gb18030', errors='ignore') as file:
    teseract = file.readlines()
teseract = [x.strip() for x in teseract] 
teseract = [x.lower() for x in teseract]
teseract_str = ' '.join(teseract)
for i in range(10):
    teseract_str=teseract_str.replace(str(i),'')
teseract_clean = re.sub(r'[{}]'.format(string.punctuation), '', teseract_str).split()


In [113]:
#fname = 'CSW_corpus.txt'
#with open(fname) as file:
 #   corpus = file.readlines()
#corpus = [x.strip() for x in corpus]   
# delete first 2 rows
#corpus = corpus[2:]
# de-capitalize
#corpus = [x.lower() for x in corpus]
corpus = set(truth_clean)
corpus

{'a',
 'about',
 'access',
 'acroosteolysis',
 'act',
 'acting',
 'action',
 'actions',
 'activities',
 'ad',
 'addition',
 'additional',
 'adequate',
 'administration',
 'administrations',
 'administrator',
 'administrators',
 'adopted',
 'adoption',
 'advertising',
 'advisors',
 'affairs',
 'affected',
 'aflcio',
 'after',
 'afterward',
 'agency',
 'agreement',
 'agree颅',
 'ahead',
 'air',
 'alerted',
 'all',
 'alleged',
 'allow',
 'allowance',
 'allowing',
 'also',
 'alter颅',
 'amendments',
 'american',
 'among',
 'an',
 'and',
 'annual',
 'anticipated',
 'any',
 'appears',
 'applied',
 'appropriate',
 'approval',
 'approved',
 'april',
 'are',
 'argument',
 'as',
 'aspects',
 'association',
 'associations',
 'assurances',
 'as颅',
 'at',
 'atlantic',
 'attaches',
 'attaching',
 'attacked',
 'attempt',
 'attended',
 'authored',
 'authorized',
 'b',
 'bags',
 'bank',
 'banking',
 'basis',
 'be',
 'been',
 'before',
 'began',
 'begin',
 'begun',
 'behalf',
 'being',
 'believe',
 'betwe

In [114]:
import numpy as np
import pandas as pd
from collections import Counter 
from nltk import edit_distance

def typo_classification(typo,correct):
    if (len(typo) > len(correct)):
        return 'insertion'
    elif (len(typo) < len(correct)):
        return 'deletion'
    else:
        typo_count = Counter(typo)
        correct_count = Counter(correct)
        if typo_count == correct_count:
            return 'reversal'
        else:
            return 'substitution'

def find_candidates(typo,corpus):
    candidates = []
    candi_type = []
    for word in corpus:
        ed = edit_distance(typo,word)
        word_type = typo_classification(typo,word)
        if ((ed == 1) | ((ed == 2) & (word_type == 'reversal'))):
            candidates.append(word)
            candi_type.append(word_type)
    return candidates,candi_type

def find_position(typo,candidates):
    position = []
    for corr in candidates:
        typo_type = typo_classification(typo,corr)
        
        if (typo_type == 'deletion'):
            typo += '#'

            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    if corr[i] != corr[i-1]:
                        typo = typo[:-1]
                        position.append([typo,corr,"@",corr[i],i,typo_type])
                        break
                    else:
                        typo = typo[:-1]
                        position.append([typo,corr,"@",corr[i],i,typo_type])
                        position.append([typo,corr,"@",corr[i],i-1,typo_type])
                        break
                        
                i += 1
        elif (typo_type == 'insertion'):
            corr += '#'

            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    if typo[i] != typo[i-1]:
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        break
                    elif ((typo[i] == typo[i-1])& (typo[i] == typo[i-2])):
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        position.append([typo,corr,typo[i],"@",i-1,typo_type])
                        position.append([typo,corr,typo[i],"@",i-2,typo_type])
                        break
                    else:
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        position.append([typo,corr,typo[i],"@",i-1,typo_type])
                        break
                i += 1
        elif (typo_type == 'substitution'):
            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    position.append([typo,corr,typo[i],corr[i],i,typo_type])
                    break
                i+=1
                
        elif (typo_type == 'reversal'):
            i = 0
            while i < len(corr)-1:
                if ((typo[i] == corr[i+1]) & (typo[i+1] == corr[i])):
                    typo_comb = typo[i] + typo[i+1]
                    position.append([typo,corr,typo_comb,typo_comb[::-1],i,typo_type])
                    break
                i +=1
    return position

In [121]:
len(truth_clean)

1672

In [122]:
len(teseract_clean)

1672

In [123]:
df1 = pd.DataFrame(truth_clean)

df1[1] = teseract_clean
# import the typos 
wrong = df1[df1[0] != df1[1]]

In [136]:
# input lowercase typo
typo = wrong[1][6]

candidates,cand_type = find_candidates(typo,corpus)
correction = find_position(typo,candidates)
wrong

,0,1
0,exhibit,xansm
1,d,u
2,staff,report
3,report,ma
4,ma,rah
5,rch,by
6,by,willlam
7,william,j
8,j,drlver
9,driver,congressional


In [137]:
correction = pd.DataFrame(correction)
correction.columns = ['Typo','Correction','old','new','index','type']
correction

,Typo,Correction,old,new,index,type
0,willlam,william,l,i,4,substitution


In [138]:
#confusiondel=pd.read_csv('C:/Users/Alienware/Desktop/ads project1/Spring2019-Proj4-grp6/data/confusion_matrix/add_matrix.csv')
confusionsub=pd.read_csv('C:/Users/Alienware/Desktop/ads project1/Spring2019-Proj4-grp6/data/confusion_matrix/sub_matrix.csv')
confusionadd=pd.read_csv('C:/Users/Alienware/Desktop/ads project1/Spring2019-Proj4-grp6/data/confusion_matrix/add_matrix.csv')
confusiondel=pd.read_csv('C:/Users/Alienware/Desktop/ads project1/Spring2019-Proj4-grp6/data/confusion_matrix/del_matrix.csv')
confusionrev=pd.read_csv('C:/Users/Alienware/Desktop/ads project1/Spring2019-Proj4-grp6/data/confusion_matrix/rev_matrix.csv')

In [24]:
correction['probability of t given c']='defalut value'
correction['probability of c']='defalut value'

,Unnamed: 0,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
26,#,46,8,9,8,26,11,14,3,5,...,0,6,23,2,11,1,2,1,1,2


In [140]:
#def probabilityfunction(type):
for i in range(0,correction.shape[0]):
    index=correction.iloc[i,4]
    specificword=correction.iloc[i,1]
    if correction.iloc[i,5]=='insertion':
        if index != 0:
            
            #index=correction.iloc[i,4]
            X=specificword[index-1]
            Y=specificword[index]
            haha=confusionadd[confusionadd.iloc[:,0]==X]
            add=haha[Y].iloc[0]
            total=0
            for z in range(0,len(truth_clean)):
                total=total+truth_clean[z].count(X+Y)
                #lis.append(total)
                correction.iloc[i,6]=add/total
        if index == 0:
            X='#'
            Y=specificword[index]
            haha=confusionadd[confusionadd.iloc[:,0]==X]
            add=haha[Y].iloc[0]
            totall=len(truth_clean)
            
            correction.iloc[i,6]=add/totall
                
    if correction.iloc[i,5]=='deletion':
        if index != 0:
            
            #index=correction.iloc[i,4]
            X=specificword[index-1]
            Y=specificword[index]
            haha=confusiondel[confusiondel.iloc[:,0]==X]
            delt=haha[Y].iloc[0]
            total=0
            for z in range(0,len(truth_clean)):
                total=total+truth_clean[z].count(X+Y)
                #lis.append(total)
                correction.iloc[i,6]=delt/total
            
            
        if index == 0:
            X='#'
            Y=specificword[index]
            haha=confusiondel[confusiondel.iloc[:,0]==X]
            delt=haha[Y].iloc[0]
            totall=len(truth_clean)
            
            correction.iloc[i,6]=delt/totall
    if correction.iloc[i,5]=='reversal':
        
            
            #index=correction.iloc[i,4]
            X=specificword[index]
            Y=specificword[index+1]
            haha=confusionrev[confusionrev.iloc[:,0]==X]
            rev=haha[Y].iloc[0]
            total=0
            for z in range(0,len(truth_clean)):
                total=total+truth_clean[z].count(X+Y)
                #lis.append(total)
                correction.iloc[i,6]=rev/total
        
                
    if correction.iloc[i,5]=='substitution':
        X=correction.iloc[i,2]
        Y=correction.iloc[i,3]
        heihei=confusionsub[confusionsub.iloc[:,0]==X]
        sub=heihei[Y].iloc[0]
        #lis.append(sub)

        total=0
        for z in range(0,len(truth_clean)):
            total=total+truth_clean[z].count(Y)
            #lis.append(total)
            correction.iloc[i,6]=sub/total
        #return correction['probability of t given c']

In [12]:
N=len(truth_clean)+len(teseract_clean)
V=len(set(truth_clean))+len(set(teseract_clean))
for a in range(0,correction.shape[0]):
    cor=correction.iloc[a,1]
    correction.iloc[a,7]=((truth_clean.count(cor)+teseract_clean.count(cor)) + 0.5)/(N + V/2)



In [15]:
correction.head()
    


,Typo,Correction,old,new,index,type,probability of t given c,probability of c
0,has,was,h,w,0,substitution,0.0185185,0.00334294
1,has,had,s,d,2,substitution,0.087766,0.000576369
2,has,as,h,@,0,insertion,defalut value,0.00518732
3,has,his,a,i,1,substitution,0.1475,0.000806916


In [87]:
freq = [] # the number of times that the proposed correction c appears in the training set
for cor in correction['Correction']:
    freq.append(len(wrong[wrong[0] == cor]))


In [9]:
N = len(truth_clean) # the number of words
V = len(set(truth_clean)) # the vocabulary size 

prior = (pd.DataFrame(freq) + 0.5)/(N + V/2)
prior

,0
0,0.000696
